In [22]:
import pandas as pd

# Carrega dataset final
df = pd.read_csv('dataset_musical_final_per_viz.csv')

# Processament de la data (Extreure l'any i la dècada)
df['year_extracted'] = df['track_album_release_date'].str[:4].astype(float)
df['decade'] = (df['year_extracted'] // 10 * 10).astype(int)

print("Anàlisi preguntes")

# P 1: L'Estereotip Emocional (Valence)
# Comprovem si les dones realment tenen una valence (felicitat) diferent
valence_analysis = df.groupby('artist_class')['valence'].agg(['mean', 'std', 'count']).sort_values(by='mean')
print("\n1. Anàlisi de Valence (Positivitat):")
print(valence_analysis)

# P 2: (Energy & Loudness)
energy_analysis = df.groupby('artist_class')[['energy', 'loudness']].mean()
print("\n2. Anàlisi d'Energia i Volum (Loudness):")
print(energy_analysis)

# P 3: Evolució Temporal
# Com ha variat la presència femenina per dècada?
temp_evolution = pd.crosstab(df['decade'], df['artist_class'], normalize='index') * 100
print("\n3. Evolució del % de Gènere per Dècada (Darrers 50 anys):")
# Mostrem només des dels 70 fins ara per claredat
print(temp_evolution.loc[1970:].round(2))

Anàlisi preguntes

1. Anàlisi de Valence (Positivitat):
                                                        mean       std  count
artist_class                                                                 
unknown                                             0.334090  0.242625    321
female                                              0.464354  0.257982    439
error: caused by: <urlopen error [WinError 1005...  0.467574  0.266214     34
male                                                0.487752  0.258611   1521
collaboration                                       0.489466  0.246348   1633
band                                                0.521043  0.264961    882

2. Anàlisi d'Energia i Volum (Loudness):
                                                      energy   loudness
artist_class                                                           
band                                                0.640006  -9.145100
collaboration                                       0.633761  

Eliminarem els errors de l'API i conservarem els Unknowns

In [23]:
# Eliminem només els errors tècnics (els 34 casos de connexió) i mantenim 'unknown' (els 321 casos)
df_final = df[~df['artist_class'].str.contains('error', na=False, case=False)].copy()

# Assegurem que la data estigui neta per al gràfic temporal
df_final['year'] = df_final['track_album_release_date'].str[:4]

# Guardem el fitxer per a la visualització
df_final.to_csv('spotify_data_FINAL_TO_VIZ.csv', index=False)

print("Fitxer 'spotify_data_FINAL_TO_VIZ.csv' llest per a la fase visual!")
print(df_final['artist_class'].value_counts())

Fitxer 'spotify_data_FINAL_TO_VIZ.csv' llest per a la fase visual!
artist_class
collaboration    1633
male             1521
band              882
female            439
unknown           322
Name: count, dtype: int64


# EDA Extens

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Carreguem el dataset definitiu
df = pd.read_csv('spotify_data_FINAL_TO_VIZ.csv')

# BLOC 1: DESCRIPCIÓ DE LA MOSTRA I OUTLIERS
# Objectiu: Entendre els extrems de la base de dades.
print("BLOC 1: Més enllà dels valors mitjans")
# Identifiquem les cançons més "extremes" per donar context al vídeo
sadest_song = df.loc[df['valence'].idxmin()][['track_name', 'track_artist', 'valence']]
happiest_song = df.loc[df['valence'].idxmax()][['track_name', 'track_artist', 'valence']]

print(f"Cançó més trista (Low Valence): {sadest_song['track_name']} de {sadest_song['track_artist']} ({sadest_song['valence']})")
print(f"Cançó més feliç (High Valence): {happiest_song['track_name']} de {happiest_song['track_artist']} ({happiest_song['valence']})")

# BLOC 2: L'estereotip emocional
# Objectiu: Respondre si les dones canten cançons més "tristes".
print("\n BLOC 2: Categorització original per gènere")

# Creem categories basades en la Valence
def get_sentiment(v):
    if v < 0.33: return 'Sad/Melancholic'
    if v < 0.66: return 'Neutral'
    return 'Happy/Cheerful'

df['sentiment'] = df['valence'].apply(get_sentiment)

# Calculem el % de cada sentiment per classe d'artista
sentiment_dist = pd.crosstab(df['artist_class'], df['sentiment'], normalize='index') * 100
print("Distribució emocional (%) per classe:")
print(sentiment_dist.round(2))

# BLOC 3: Loudness i energy
# Objectiu: Veure si la música s'ha tornat més "agressiva" sonorament.
print("\nBLOC 3: Agressivitat i energia sonora")

# Mirem com ha canviat el volum (loudness) per dècada
loudness_war = df.groupby('decade')['loudness'].mean().sort_index()
print("Evolució del volum mitjà (dB) per dècada:")
print(loudness_war)


# BLOC 4: L'Indie vs mainstream
# Objectiu: Validar si els artistes no detectats són realment més "orgànics".
print("\nBLOC 4: El perfil més orgànic deslñ unknown")

# Comparem Acousticness i Instrumentalness
# Un valor alt en ambdues indica música menys processada electrònicament
organic_profile = df.groupby('artist_class')[['acousticness', 'instrumentalness', 'liveness']].mean()
print(organic_profile)

# Correlació entre Popularitat i Artificialitat
# Veiem si ser menys instrumental ajuda a ser més popular segons la classe
print("\nCorrelació Popularitat vs Acousticness (Biaix comercial):")
for cls in ['male', 'female', 'collaboration', 'band', 'unknown']:
    subset = df[df['artist_class'] == cls]
    corr = subset['track_popularity'].corr(subset['acousticness'])
    print(f"{cls}: {corr:.2f}")

# BLOC 5: Correlació de l'èxit amb danceability
# Objectiu: Respondre si la ballabilitat correlaciona amb l'èxit femení.
print("\nBLOC 5: La ballabilitat")

# Mirem quina classe d'artista ha capitalitzat millor el ritme ballable
dance_success = df.groupby('artist_class')[['danceability', 'track_popularity']].mean()
print(dance_success)

BLOC 1: Més enllà dels valors mitjans
Cançó més trista (Low Valence): Somnova de Reso Nata (0.0296)
Cançó més feliç (High Valence): Stop de B.W.H. (0.987)

 BLOC 2: Categorització original per gènere
Distribució emocional (%) per classe:
sentiment      Happy/Cheerful  Neutral  Sad/Melancholic
artist_class                                           
band                    34.69    38.32            26.98
collaboration           28.54    43.72            27.74
female                  26.65    39.64            33.71
male                    28.86    39.58            31.56
unknown                 11.49    35.09            53.42

BLOC 3: Agressivitat i energia sonora
Evolució del volum mitjà (dB) per dècada:
decade
1950   -10.985000
1960    -9.426889
1970    -9.955857
1980    -7.986087
1990    -9.392869
2000    -7.786170
2010    -8.094616
2020    -9.892941
Name: loudness, dtype: float64

BLOC 4: El perfil més orgànic deslñ unknown
               acousticness  instrumentalness  liveness
artist

## Bloc 3: Anàlisi de la Paradoxa de Diversitat (Gènere vs. Ballabilitat)
En aquest apartat preparem les dades per analitzar si l'increment del ritme i la ballabilitat ha afectat directament la quota de mercat de les dones solistes.

In [26]:
import pandas as pd

# Carreguem el dataset
df = pd.read_csv('spotify_data_FINAL_TO_VIZ.csv')

# Preparem les dades per dècada (des de l'any 2000)
df_2000 = df[df['decade'] >= 2000].copy()

# % de presència femenina
presencia = pd.crosstab(df_2000['decade'], df_2000['artist_class'], normalize='index') * 100
female_presence = presencia['female']

# Ballabilitat mitjana
dance_avg = df_2000.groupby('decade')['danceability'].mean()

# Càlcul base 0 (Percentatge de variació)
# La fòrmula: ((Valor actual / Valor inicial) - 1) * 100
# Això fa que l'any 2000 valgui exactament 0.

df_base0 = pd.DataFrame({
    'Dones_Variacio_%': ((female_presence / female_presence.iloc[0]) - 1) * 100,
    'Danceability_Variacio_%': ((dance_avg / dance_avg.iloc[0]) - 1) * 100
}).reset_index()

# Guardem el fitxer
df_base0.to_csv('grafica_3_base_zero.csv', index=False)
print(df_base0)

   decade  Dones_Variacio_%  Danceability_Variacio_%
0    2000          0.000000                 0.000000
1    2010        -26.802945                 5.864499
2    2020        -36.717146                 8.617366


# Slides extra

In [27]:
import pandas as pd

# Carreguem el dataset principal
df = pd.read_csv('spotify_data_FINAL_TO_VIZ.csv')

# Context del mercat 
# Volem saber quin % del pastís té cada grup per dècada
market_share = pd.crosstab(df['decade'], df['artist_class'], normalize='index') * 100

# Calculem les mètriques de format (Durada i Popularitat)
# Passem la durada de mil·lisegons a segons per fer-ho més llegible
df['duration_sec'] = df['duration_ms'] / 1000
format_trends = df.groupby('decade').agg({
    'duration_sec': 'mean',
    'loudness': 'mean',
    'track_popularity': 'mean'
})

# juntem tot en un únic dataset extra
df_extra = pd.concat([market_share, format_trends], axis=1).reset_index()

# Guardem el fitxer per a les slides extres
df_extra.to_csv('dades_extra_analisi.csv', index=False)
print(df_extra.head())

   decade       band  collaboration     female        male   unknown  \
0    1950   0.000000       0.000000   0.000000  100.000000  0.000000   
1    1960  66.666667       0.000000  11.111111   22.222222  0.000000   
2    1970  60.714286       8.928571   1.785714   26.785714  1.785714   
3    1980  64.077670       2.912621   9.708738   23.300971  0.000000   
4    1990  51.492537      13.805970   4.477612   30.223881  0.000000   

   duration_sec   loudness  track_popularity  
0    161.473000 -10.985000         74.500000  
1    207.289222  -9.426889         75.777778  
2    264.377089  -9.955857         73.982143  
3    260.476796  -7.986087         74.291262  
4    273.484541  -9.392869         60.216418  


In [28]:
import pandas as pd

### Slide tiktok

# Carreguem les dades
df = pd.read_csv('spotify_data_FINAL_TO_VIZ.csv')
df['year'] = pd.to_numeric(df['track_album_release_date'].str[:4], errors='coerce')
df['duration_sec'] = df['duration_ms'] / 1000

# Filtrem des de l'any 2000 i agrupem per any
resum_anual = df[df['year'] >= 2000].groupby('year').agg({
    'duration_sec': 'mean',
    'loudness': 'mean'
})

# Calculem el canvi respecte a l'any 2000 (Base 0)
# Formula: ((Valor Any X / Valor Any 2000) - 1) * 100
any_2000_dur = resum_anual.loc[2000, 'duration_sec']
any_2000_vol = resum_anual.loc[2000, 'loudness']

resum_anual['Canvi Durada (%)'] = ((resum_anual['duration_sec'] / any_2000_dur) - 1) * 100
resum_anual['Canvi Volum (%)'] = ((resum_anual['loudness'] / any_2000_vol) - 1) * 100

# Guardem el fitxer per a Flourish
resum_anual.reset_index()[['year', 'Canvi Durada (%)', 'Canvi Volum (%)']].to_csv('viz_tiktok_anual.csv', index=False)